## Imports

In [1]:
import glob
import pandas as pd
import re
import numpy as np
import nltk
import math

#nltk.download()

## Functions

### Save

In [2]:
def save_as_csv(filename, dataframe) :
    dataframe.to_csv(filename, encoding='utf-8')

### Read all csv

In [3]:
def readcsv(path):
    data_input = pd.read_csv(path, header=0, usecols=range(1,3))
    data_frame_input = pd.DataFrame(data = data_input)
    return data_frame_input
formal_words_data = readcsv("text_data/wordlist2.csv")

In [4]:
def is_contain(check_data, datas):
    i = 0
    while ((i < len(datas)) and (check_data[0] != datas[i][0])) :
        i = i + 1
    return (i < len(datas))

def filter_same_data(check_datas, datas):
    new_array = datas
    for i in range(len(check_datas)) :
        if (not(is_contain(check_datas[i], new_array))):
            if (len(new_array) == 0):
                new_array = [check_datas[i]]
            else :
                new_array = np.append(new_array,[check_datas[i]], axis = 0)
    new_array = pd.DataFrame(data = new_array).values
    return (new_array)

def read_all_csv():
    result = []
    attribute_names = ["","tweet","user"]
    csvs_path = glob.glob('raw_data/*.csv')
    for path in csvs_path :
        data_input = pd.read_csv(path, header=0, usecols=range(1,3))
        data_frame_input = pd.DataFrame(data = data_input).values
        apply_first_normalization(data_frame_input)
        result = filter_same_data(data_frame_input, result)
    data_frame_result = pd.DataFrame(data = result)
    return (data_frame_result.values)

### Filter Emoji

In [5]:
def filter_emoji_in_array(datas):
    for i in range(len(datas)):
        datas[i][0] = filter_emoji(datas[i][0])

def filter_emoji(text):
    emoji_pattern = re.compile("["
        u"\U00002700-\U000027BF"  # dingbats
        u"\U00002460-\U0001F2FF"  # enclosed characters
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F910-\U0001F9FF"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U00000080-\U000000FF"  # uncategorized 1 (latin 1 supplement)
        u"\U00002000-\U0000206F"  # uncategorized 2 (general punctuation)
        #u"\U00000000-\U0000007F"  # uncategorized 3 (basic latin)
        u"\U000020D0-\U000020FF"  # uncategorized 4 (Combining Diacritical Marks for Symbols )
        u"\U00002100-\U0000214F"  # uncategorized 5 (Letterlike Symbols )
        u"\U00002300-\U000023FF"  # uncategorized 6 (Misc Technical )
        u"\U000025A0-\U000025FF"  # uncategorized 7 (Geometric Shapes )
        u"\U00002600-\U000026FF"  # uncategorized 8 (Misc Symbols )
        u"\U00002900-\U0000297F"  # uncategorized 9 (Supp Arrows B )
        u"\U00002B00-\U00002BFF"  # uncategorized 10 (Misc Symbol Arrows )
        u"\U00003000-\U0000303F"  # uncategorized 11 (CJK Sumbols and Punc )
        u"\U00003200-\U000032FF"  # uncategorized 12 (Enclosed CJK Letters and Months)
        u"\U0001F000-\U0001F02F"  # uncategorized 13 (Mahjong Tiles)
        u"\U0001F0A0-\U0001F0FF"  # uncategorized 14 (Playing Cards)
        u"\U0001F300-\U0001F5FF"  # uncategorized 15 (Misc Symbols and Pict)
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    return(emoji_pattern.sub(r'', text))


### Lowercase

In [6]:
def lowercase_array(datas):
    result = datas
    for i in range(len(datas)):
        result[i][0] = datas[i][0].lower()
    datas = result

In [7]:
def apply_first_normalization(datas):
    filter_emoji_in_array(datas)
    lowercase_array(datas)

### Filter Non Words

In [8]:
def filter_url(text):
    return(re.sub(r"http\S+", "", text))

def filter_tag(text):
    return(re.sub(r"@\S+", "", text))

def filter_hashtag(text):
    return(re.sub(r"#\S+", "", text))

def filter_not_alphanumeric(text):
    return(re.sub(r'[^%\w]', ' ', text))

def filter_number(text):
    return(re.sub(r'[0-9]', '', text))

In [9]:
def filter_non_words(datas):
    result = datas.copy(deep=True)
    for i in range(len(datas)):
        result[0][i] = filter_url(result[0][i])
        result[0][i] = filter_tag(result[0][i])
        result[0][i] = filter_hashtag(result[0][i])
        result[0][i] = filter_not_alphanumeric(result[0][i])
        #result[0][i] = filter_number(result[0][i])
    return result

### Split Words

In [10]:
def words_list_array(datas):
    result = []
    for i in range(len(datas)):
        new_datas = nltk.word_tokenize(datas[0][i])
        new_datas = pd.DataFrame(data=new_datas).values
        result = filter_same_data(new_datas, result)
        #result.extend(new_datas)
    result = pd.DataFrame(data=result)
    return result

def words_list(data):
    return(nltk.word_tokenize(data))

In [11]:
def get_same_data_index(check_data, datas):
    i = 0
    while ((i < len(datas)) and (check_data != datas[i])) :
        i = i + 1
    if (i< len(datas)):
        return i
    else:
        return -1

def formalize_sentence(raw_sentence, formal_words):
    raw_words = nltk.word_tokenize(raw_sentence)
    result = ""
    for i in range(len(raw_words)):
        index = get_same_data_index(raw_words[i], formal_words["0"])
        if (index!= -1):
            if (type(formal_words["1"][index]) != float):
                raw_words[i] = formal_words["1"][index]
        result = result + " " + str(raw_words[i])
    return result

def formalize_array_sentence(array_raw_sentence, formal_words):
    result = array_raw_sentence.copy(deep=True)
    for i in range(len(result)):
        result[0][i] = formalize_sentence(result[0][i],formal_words)
        #print(result[0][i])
    return result

#print(formalize_sentence("cpt yg bs jangan", formal_words_data))

## Main

In [12]:
raw_data = read_all_csv()
raw_data = pd.DataFrame(data=raw_data)
save_as_csv("integrated_data/integrated.csv",raw_data)

print(len(raw_data))
print(raw_data)

1344
                                                      0                1
0      ya anjir cpt bgt ya abis barang manusia diskon:(         SOHEEDCH
1     lazada 11.11.2019 diskon terbesar 24 jam https...        Vallno_07
2     selama kalo liat promo diskon 70% + 30% dalem ...         bayu_joo
3     apa bedanya matahari sama bulan?  matahari ada...       Hanifati_f
4     @garudacares bagaimana cara tukar mileage disk...           Nug_QA
5     @gencamon nahiya benerrr hahahaha kalo makan c...         primetam
6            fak gmarket kenapa diskon nya sekarang sih       singularyv
7     sejarah asia tenggara karya m.c. ricklefs rp 3...   KomunitasBambu
8     @amingcoffee transmart kuburaya diskon, kapan ...      kalbarinfo_
9     tah diskon 50% cenah..  yuk diserbu..   https:...   AmaAbiyyunShoh
10    @wongalas901 @prabowo @aniesbaswedan rika wis ...     Dianhandoko6
11    sebenarnya bahagia itu sederhana... - beli ini...        anggieppu
12    yuhuu..  siapa lagi yang mau order deodo

In [13]:
only_words_datas = filter_non_words(raw_data)

print(only_words_datas)

                                                      0                1
0      ya anjir cpt bgt ya abis barang manusia diskon           SOHEEDCH
1             lazada 11 11 2019 diskon terbesar 24 jam         Vallno_07
2     selama kalo liat promo diskon 70%   30% dalem ...         bayu_joo
3     apa bedanya matahari sama bulan   matahari ada...       Hanifati_f
4      bagaimana cara tukar mileage diskon 70%  garu...           Nug_QA
5      nahiya benerrr hahahaha kalo makan cantik ken...         primetam
6            fak gmarket kenapa diskon nya sekarang sih       singularyv
7     sejarah asia tenggara karya m c  ricklefs rp 3...   KomunitasBambu
8      transmart kuburaya diskon  kapan lagi rp 5 00...      kalbarinfo_
9              tah diskon 50% cenah    yuk diserbu        AmaAbiyyunShoh
10       rika wis inden mobil esemka nek arp tuku ga...     Dianhandoko6
11    sebenarnya bahagia itu sederhana      beli ini...        anggieppu
12    yuhuu    siapa lagi yang mau order deodorantn

In [14]:
word_list_datas = words_list_array(only_words_datas)
save_as_csv("text_data/wordlist.csv",word_list_datas)

print(len(word_list_datas))
print(word_list_datas)

5517
                 0
0               ya
1            anjir
2              cpt
3              bgt
4             abis
5           barang
6          manusia
7           diskon
8           lazada
9               11
10            2019
11        terbesar
12              24
13             jam
14          selama
15            kalo
16            liat
17           promo
18              70
19               %
20              30
21           dalem
22            hati
23        langsung
24          bilang
25             ini
26            jadi
27          gratis
28            dong
29             itu
...            ...
5487      nunjukin
5488          malu
5489       sekolah
5490      belinyaa
5491      dimaanaa
5492           opp
5493          road
5494     rayainnya
5495     terpotong
5496        proses
5497            sg
5498           feb
5499  wekawekaweka
5500         laahh
5501           130
5502          olih
5503         namba
5504          memb
5505      nganyang
5506           24k
5507   

In [15]:
raw_words_data = only_words_datas.head(200)

formal_datas = formalize_array_sentence(raw_words_data, formal_words_data)
save_as_csv("integrated_data/normalized.csv",formal_datas)

print(formal_datas)

                                                     0                1
0     ya anjir cepat sekali ya habis barang manusia...         SOHEEDCH
1             lazada 11 11 2019 diskon terbesar 24 jam        Vallno_07
2     selama kalau lihat promo diskon 70 % 30 % dal...         bayu_joo
3     apa bedanya matahari sama bulan matahari ada ...       Hanifati_f
4     bagaimana cara tukar mileage diskon 70 % garu...           Nug_QA
5     iya benar ketawa kalau makan cantik kena disk...         primetam
6          f*** gmarket kenapa diskon nya sekarang sih       singularyv
7     sejarah asia tenggara karya m c ricklefs rupi...   KomunitasBambu
8     transmart kuburaya diskon kapan lagi rupiah 5...      kalbarinfo_
9                      nah diskon 50 % lho yuk diserbu   AmaAbiyyunShoh
10    rika sudah inden mobil esemka kalau hendak be...     Dianhandoko6
11    sebenarnya bahagia itu sederhana beli ini bel...        anggieppu
12    yuk siapa lagi yang mau order deodoran deodor...  indri_in